# Notebook 16: Expectation Maximization in practice


## Learning Goal 
The goal of this notebook is to gain intuition for Expectation Maximization using a simple example involving coin tosses.

## Overview

In Section XIV, we introduce Expectation-Maximization (EM) as a practical way to perform maximum likelihood estimation (MLE) even when some of the data is hidden (i.e in the presence of latent or hidden variables). To better understand EM, in this short notebook we'll explore a very simple coin-tossing example adapted from [Do and Batzoglou, Nat. Biotechnol. (2008)](https://www.nature.com/articles/nbt1406). 

Suppose that we are given two coins A and B with unkown bias $\theta_A$ and $\theta_B$, respectively. Our goal is to estimate the bias vector $\boldsymbol{\theta}= (\theta_A, \theta_B)$ from the outcomes of the following experiment: 

<blockquote> 
First choose one coin at random. Then toss the selected coin 10 times independently and record the number of heads observed. Repeat this procedure 5 times.
</blockquote>

Formally, let $z_i\in\{A,B\}$ be the coin selected in experiment $i$ and $x_i\in\{0,1,\cdots 10\}$ be the number heads recorded by tossing $z_i$ 10 times. Since we conduct $n=5$ such experiments, we can summarize the outcomes of these 50 tosses by two vectors: $\boldsymbol{x}=(x_1,x_2\cdots, x_5)$ and $\boldsymbol{z}=(z_1,z_2,\cdots, z_5)$.




### Exercise 1: What if we know everything?

 * Consider first the case where we have complete knowledge of the experiment, namely, both $\boldsymbol{x}$ and $\boldsymbol{z}$ are known. How would you intuitively estimate the biases of the two coins  $\boldsymbol{\theta}= (\theta_A, \theta_B)$ ?

#### We would flip the coins N times and record the number of heads, Nhead. The bias would then be NHead/N
 * What's the likelihood of observing the complete outcomes of these experiments? In other words, what is $P(\boldsymbol{x},\boldsymbol{z}| n,\boldsymbol{\theta} )$? You may assume this is a Bernoulli trial. Namely, every time coin A(B) is tossed, we have, with probability $\theta_A$($\theta_B$), that the outcome is heads.
 
 * What's the Maximum Likelihood Estimator (MLE)? Is this consistent with your intuition? 
 

For 1 coin which we have flipped n times where we have observed k heads and n-k tails the Bernouli probability of observing this outcome is:

$P(\theta_{A}) = C(n,k)*\theta_{A}^{k}*(1-\theta_{A})^{n-k}$

The loglikelihood is:

$log(P(\theta_{A})) = log(C(n,k)) + k*log(*\theta_{A}) + (n-k)*log(1-\theta_{A})$

We maximixe the loglikelihood by setting

$\frac{d(log(P(\theta_{A})))}{d\theta_{A}} = 0$

Solving for $\theta_{A}$ we get:
$\theta_{A}=k/n$

## Comparing MLE and EM

To test your answer, let's do some numerics! We will compare the MLE estimates of biases with an Expectation Maximization procedure where we do not know ${\bf z}$. The following code computes our best guess for the biases using MLE -- assuming we know the identity of the coin used -- and compares it estimates arrived at using an EM procedure where we have no knowledge about which coin was being tossed (though we know the same coin was tossed 10 times).

In [2]:
import numpy as np
from scipy.special import comb
import math


def compute_likelihood(obs, n, pheads): # No surprise, it's Binomial!!!

    likelihood = comb(n, obs, exact=True)*(pheads**obs)*(1.0-pheads)**(n-obs)

    return likelihood

# generate experiments
num_coin_toss = 10 # each experiment contains num_coin_toss tosses
num_exp = 5  # we perform 5 such experiments
theta_A_true = 0.8 
theta_B_true = 0.4
coin_choice = np.zeros(num_exp) # initialize: 0 for A and 1 for B
head_counts = np.zeros(num_exp)

# MLE 
MLE_A = 0.0
MLE_B = 0.0

# generate the outcomes of experiment
for i in np.arange(num_exp):
    
    if np.random.randint(2) == 0: # coin A is selected
        head_counts[i] = np.random.binomial(num_coin_toss , theta_A_true, 1) # toss coin A num_coin_toss times
        MLE_A = MLE_A +  head_counts[i] # add the number of heads observed to total headcounts 
    
    else: # coin B is selected 
        head_counts[i] = np.random.binomial(num_coin_toss , theta_B_true, 1) # toss coin B num_coin_toss times
        coin_choice[i] = 1  # record the selection of coin B during experiment i 
        MLE_B = MLE_B +  head_counts[i] # add the number of heads observed to total headcounts 
    
tail_counts = num_coin_toss - head_counts


# MLE is merely the proportion of heads for each coin toss
MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice))*num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice)*num_coin_toss)



# initialize the pA(heads) and pB(heads), namely, coin biases
pA_heads = np.zeros(100); 
pB_heads = np.zeros(100); 

pA_heads[0] = 0.60 # initial guess
pB_heads[0] = 0.50 # initial guess

# E-M begins!
epsilon = 0.001   # error threshold
j = 0 # iteration counter
improvement = float('inf')

while (improvement > epsilon):
    
    expectation_A = np.zeros((num_exp,2), dtype=float) 
    expectation_B = np.zeros((num_exp,2), dtype=float)
    
    for i in np.arange(min(len(head_counts),len(tail_counts))):
        
        eH = head_counts[i]
        eT = tail_counts[i]
        
        # E step:
        lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
        lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])
        
        weightA = lA / (lA + lB)
        weightB = lB / (lA + lB)
              
        expectation_A[i] = weightA*np.array([eH, eT])
        expectation_B[i] = weightB*np.array([eH, eT])

  
    # M step
    theta_A = np.sum(expectation_A, axis = 0)[0] / np.sum(expectation_A) 
    theta_B = np.sum(expectation_B, axis = 0)[0] / np.sum(expectation_B) 

    print('At iteration %d, theta_A = %2f,  theta_B = %2f' % (j, theta_A, theta_B))
    
    pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A)); 
    pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B)); 

    improvement = max( abs(np.array([pA_heads[j+1],pB_heads[j+1]]) - np.array([pA_heads[j],pB_heads[j]]) ))
    j = j+1

# END of E-M, print the outcome

print('E-M converges at iteration %d' %j)
print('RESULT:')
print('E-M: theta_A = %2f,  theta_B = %2f' % (theta_A, theta_B))
print('MLE with complete data: theta_A = %2f,  theta_B = %2f' % (MLE_A, MLE_B))

At iteration 0, theta_A = 0.627295,  theta_B = 0.336641
At iteration 1, theta_A = 0.757067,  theta_B = 0.240138
At iteration 2, theta_A = 0.796721,  theta_B = 0.233501
At iteration 3, theta_A = 0.798918,  theta_B = 0.233551
At iteration 4, theta_A = 0.798990,  theta_B = 0.233556
E-M converges at iteration 5
RESULT:
E-M: theta_A = 0.798990,  theta_B = 0.233556
MLE with complete data: theta_A = 0.800000,  theta_B = 0.233333


### Exercise 2

 * How fast does EM converge? Is the converged result close to what you'd get from MLE? 
 
 #### The EM algorithm converges within about 10 steps
 
 * Following Exercise 1, what's the objective function we're optimizing in the E-step? Does this function have a *unique global maximum*? 
 
 #### The EM algorithm guarentees that the loglikelihood will not decrease with each iteration and will eventually converge to a local maximum. The algorithm does not guarentee that this maxiumum is a global maximum
 
 * Compare both the results of MLE and EM to the actual bias (i.e. *theta_A_true*  and *theta_B_true* in the snippet above), comment on their performance.
 
#### In many cases the EM algorithm gets stuck on a local maximum

## Final remarks: a few practical tricks

From Exercise 2 and Section XIV, we know that the E-M algorithm often approximates the MLE even in the presence of latent (hidden variables). Like with most optimization methods for nonconcave functions, E-M only guarantees convergence to a local maximum of the objective function. For this reason, its performance can be boosted by running the EM procedure starting with multiple initial parameters. 

### Exercise 3

* Now instead of having a fixed initial guess of coin biases (i.e. *pA_heads[0]* and *pB_heads[0]* in the snippet), draw these values uniformly at random from $[0,1]$ and run the E-M algorithm. Repeat this twenty times and report what you observed. What's the best initial guess that gives the closest estimate to the true parameters?

* As we discussed in Section X (LinReg), **Maximum a posteriori (MAP)** estimation differs from MLE in that it employs an augmented objective function which incorporates a prior distribution over the quantities we want to estimate, and the prior distribution can be think of as a regularizer for the objective fuction used in MLE. Here we will explore how to extend E-M to MAP estimation. 

  (1) First derive the MAP estimate for the one-coin-flipping example, namely,
  $$
  \hat{{\theta}}_{MAP}(\boldsymbol{x}) = \arg\max_{\theta\in[0,1]} \log P(\boldsymbol{x}|n,{\theta} ) + \log P({\theta}),
  $$
  where 
  $$P(\boldsymbol{x}|n,{\theta}) = \prod_{i=1}^{10} \text{Binomial} (x_i|n,\theta)$$
  
  $$P({\theta})=\mathcal{N}(\theta|\mu, \sigma)$$
  
  (2) Based on (1), now modify the E-M snippet above to incorporate this prior distribution into the **M-step**. Comment on the performance. For the prior choice, try $P(\boldsymbol{\theta})=\mathcal{N}(\theta_A|0.83, 1)\mathcal{N}(\theta_B|0.37, 1)$.

In [3]:
# Generate experiments
num_coin_toss = 10
num_exp = 5
theta_A_true = 0.8
theta_B_true = 0.4
coin_choice = np.zeros(num_exp)
head_counts = np.zeros(num_exp)

MLE_A = 0.0
MLE_B = 0.0

for i in np.arange(num_exp):
    if np.random.randint(2) == 0:
        head_counts[i] = np.random.binomial(num_coin_toss, theta_A_true, 1)
        MLE_A = MLE_A + head_counts[i]
    else:
        head_counts[i] = np.random.binomial(num_coin_toss, theta_B_true, 1)
        coin_choice[i] = 1
        MLE_B = MLE_B + head_counts[i]

tail_counts = num_coin_toss - head_counts

MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice)) * num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice) * num_coin_toss)

def em_algorithm(pA_init, pB_init):
    pA_heads = np.zeros(100)
    pB_heads = np.zeros(100)

    pA_heads[0] = pA_init
    pB_heads[0] = pB_init

    epsilon = 0.001
    j = 0
    improvement = float('inf')

    while (improvement > epsilon):

        expectation_A = np.zeros((num_exp, 2), dtype=float)
        expectation_B = np.zeros((num_exp, 2), dtype=float)

        for i in np.arange(min(len(head_counts), len(tail_counts))):
            eH = head_counts[i]
            eT = tail_counts[i]

            lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
            lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])

            weightA = lA / (lA + lB)
            weightB = lB / (lA + lB)

            expectation_A[i] = weightA * np.array([eH, eT])
            expectation_B[i] = weightB * np.array([eH, eT])

        theta_A = np.sum(expectation_A, axis=0)[0] / np.sum(expectation_A)
        theta_B = np.sum(expectation_B, axis=0)[0] / np.sum(expectation_B)

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A))
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B))

        improvement = max(abs(np.array([pA_heads[j+1], pB_heads[j+1]]) - np.array([pA_heads[j], pB_heads[j]])))
        j = j + 1

    return pA_heads[j], pB_heads[j]

# Run the EM algorithm 20 times with random initial guesses
num_runs = 20
results = []

for run in range(num_runs):
    pA_init = np.random.rand()
    pB_init = np.random.rand()
     
    theta_A_est, theta_B_est = em_algorithm(pA_init, pB_init)
    print("Initial guess:", pA_init, pB_init)
    print("Estimates:", theta_A_est, theta_B_est)
    results.append((pA_init, pB_init, theta_A_est, theta_B_est))
   
#
# Find the best initial guess that gives the closest estimate to the true parameters
min_error = float('inf')
best_initial_guess = None
best_estimates = None

for res in results:
    pA_init, pB_init, theta_A_est, theta_B_est = res
    error = np.abs(theta_A_true - theta_A_est) + np.abs(theta_B_true - theta_B_est)
    
    if error < min_error:
        min_error = error
        best_initial_guess = (pA_init, pB_init)
        best_estimates = (theta_A_est, theta_B_est)

print("Best initial guess:", best_initial_guess)
print("Best estimates:", best_estimates)

Initial guess: 0.2260985967317951 0.7364310736511194
Estimates: 0.6023727723403088 0.9188832815214594
Initial guess: 0.5707474722825207 0.7646426920170252
Estimates: 0.6021191577046413 0.9182755130646625
Initial guess: 0.6584614448453109 0.45101856568657506
Estimates: 0.9183068588172036 0.6021322754501264
Initial guess: 0.10429863523455762 0.8054962726993139
Estimates: 0.602346546720025 0.9188202654101292
Initial guess: 0.8679801997847387 0.1336852355930469
Estimates: 0.9189050328898933 0.6023818209770486
Initial guess: 0.8369090405307404 0.27645925907636426
Estimates: 0.9183062182170343 0.6021320074016911
Initial guess: 0.20395142462453553 0.658082550356658
Estimates: 0.6023421236458004 0.9188096412313629
Initial guess: 0.49924742041491255 0.971313137789559
Estimates: 0.6047793038019013 0.9248032636489949
Initial guess: 0.10808790487725561 0.5903757217360169
Estimates: 0.6023079099949683 0.9187274974258557
Initial guess: 0.08702491264025702 0.6558656260480776
Estimates: 0.602306883023

Now we will modify the algorithm to include the prior distribution. The M-step will be updated to maximuze the augmented objective function with the prior distribution. 

In [6]:
import numpy as np
from scipy.special import comb
import math

def compute_likelihood(obs, n, pheads):
    likelihood = comb(n, obs, exact=True) * (pheads**obs) * (1.0-pheads)**(n-obs)
    return likelihood

def gaussian_prior(theta, mu, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((theta - mu) / sigma)**2)

# ... (previous code for generating experiments remains unchanged) ...

def em_algorithm_map(pA_init, pB_init):
    pA_heads = np.zeros(100)
    pB_heads = np.zeros(100)

    pA_heads[0] = pA_init
    pB_heads[0] = pB_init

    epsilon = 0.001
    j = 0
    improvement = float('inf')

    mu_A, sigma_A = 0.83, 1
    mu_B, sigma_B = 0.37, 1

    while (improvement > epsilon):

        # ... (previous code for E-step remains unchanged) ...

        # M step with prior
        theta_A = np.sum(expectation_A, axis=0)[0] / (np.sum(expectation_A) + (sigma_A**2) * (pA_heads[j] - mu_A))
        theta_B = np.sum(expectation_B, axis=0)[0] / (np.sum(expectation_B) + (sigma_B**2) * (pB_heads[j] - mu_B))

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A))
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B))

        improvement = max(abs(np.array([pA_heads[j+1], pB_heads[j+1]]) - np.array([pA_heads[j], pB_heads[j]])))
        j = j + 1

    return pA_heads[j], pB_heads[j]

# Run the EM algorithm 20 times with random initial guesses
num_runs = 20
results = []

for run in range(num_runs):
    pA_init = np.random.rand()
    pB_init = np.random.rand()

    theta_A_est, theta_B_est = em_algorithm_map(pA_init, pB_init)
    results.append((pA_init, pB_init, theta_A_est, theta_B_est))

# Report the results
print("Initial Guesses and MAP Estimates:")
print("Init θ_A, Init θ_B, MAP θ_A, MAP θ_B")
for result in results:
    print(f"{result[0]:.3f}, {result[1]:.3f}, {result[2]:.3f}, {result[3]:.3f}")

# Find the best initial guess that gives the closest estimate to the true parameters
min_diff = float('inf')
best_init_guess = None
best_estimates = None

for result in results:
    diff = abs(result[2] - theta_A_true) + abs(result[3] - theta_B_true)
    if diff < min_diff:
        min_diff = diff
        best_init_guess = (result[0], result[1])
        best_estimates = (result[2], result[3])

print("\nBest Initial Guess:")
print(f"θ_A: {best_init_guess[0]:.3f}, θ_B: {best_init_guess[1]:.3f}")

print("\nClosest Estimate to True Parameters:")
print(f"MAP θ_A: {best_estimates[0]:.3f}, MAP θ_B: {best_estimates[1]:.3f}")

print("\nTrue Parameters:")
print(f"θ_A: {theta_A_true}, θ_B: {theta_B_true}")

Initial Guesses and MAP Estimates:
Init θ_A, Init θ_B, MAP θ_A, MAP θ_B
0.971, 0.498, 0.799, 0.234
0.385, 0.182, 0.799, 0.234
0.824, 0.065, 0.799, 0.234
0.472, 0.183, 0.799, 0.234
0.103, 0.054, 0.799, 0.234
0.868, 0.409, 0.799, 0.234
0.409, 0.173, 0.799, 0.234
0.424, 0.943, 0.799, 0.234
0.709, 0.153, 0.799, 0.234
0.220, 0.481, 0.799, 0.234
0.280, 0.173, 0.799, 0.234
0.202, 0.848, 0.799, 0.234
0.937, 0.629, 0.799, 0.234
0.785, 0.150, 0.799, 0.234
0.666, 0.402, 0.799, 0.234
0.086, 0.770, 0.799, 0.234
0.086, 0.452, 0.799, 0.234
0.053, 0.369, 0.799, 0.234
0.957, 0.745, 0.799, 0.234
0.891, 0.902, 0.799, 0.234

Best Initial Guess:
θ_A: 0.971, θ_B: 0.498

Closest Estimate to True Parameters:
MAP θ_A: 0.799, MAP θ_B: 0.234

True Parameters:
θ_A: 0.8, θ_B: 0.4
